In [15]:
import importlib
# --------------------
from sklearn.utils import gen_batches
from sklearn.model_selection import GridSearchCV, KFold, TimeSeriesSplit
import numpy as np
from numpy.linalg import norm,inv,matrix_rank
import h5py

from Code.SGDPLS import SGDPLS
from Code.OLPLS import OLPLS
from Code.CIPLS import CIPLS
from Code.SIMPLS import SIMPLS
from Code.IPLS import IPLS
# ----------
from sklearn.cross_decomposition import PLSRegression
from pytictoc import TicToc

tictoc=TicToc()

In [12]:
with h5py.File('./data/merra2_t.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    n_train = int(n_train/4)
    print(n_train)
    
    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]
 

1103


In [14]:
# tscv = TimeSeriesSplit(n_splits=74, test_size=30)
tscv = TimeSeriesSplit(n_splits=18, test_size=30)
for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)} ~ {len(train_index)/30: .2f} months")
    print(f"  Test:  len={len(test_index)}")
    if(i>4):
        break

Fold 0:
  Train: len=563 ~  18.77 months
  Test:  len=30
Fold 1:
  Train: len=593 ~  19.77 months
  Test:  len=30
Fold 2:
  Train: len=623 ~  20.77 months
  Test:  len=30
Fold 3:
  Train: len=653 ~  21.77 months
  Test:  len=30
Fold 4:
  Train: len=683 ~  22.77 months
  Test:  len=30
Fold 5:
  Train: len=713 ~  23.77 months
  Test:  len=30


In [ ]:
tictoc=TicToc()

n_comp_lst=list(range(1,11))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(PLSRegression(), parameters,
                   cv=tscv, n_jobs=1,
                   scoring='neg_root_mean_squared_error')
print(clf)
tictoc.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tictoc.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))